# Model

First we add the necessary imports.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle

Now we declare some paths in order to produce and grab the required files.

In [ ]:
csv_file_path = "colors.csv"
color_classifier_path = "color_classifier.pth"
label_encoder_path = "label_encoder.pkl"

We can take a look at the dataset we are using to gather knowledge about the different colors in our `colors.csv` file.

In [ ]:
pd.read_csv(csv_file_path).head()

Now we define a custom PyTorch Dataset class named `ColorDataset`. A Dataset in PyTorch is an abstraction that represents a collection of data samples, where each sample typically consists of one or more input variables (e.g. an image, a text sequence, a set of features) and a corresponding output variable or label (e.g. a category, a regression target).

The ColorDataset class takes two arguments `X` and `y` in its constructor, which are the input and output data respectively. Both `X` and `y` are assumed to be arrays or lists of data samples, where `X` contains the input data and `y` contains the corresponding labels.

In the constructor, the input and output data are converted to PyTorch tensors using `torch.tensor()`. `dtype` argument specifies the data type of the tensors. Here, `X` is converted to a float32 tensor, and `y` is converted to an int64 tensor.

The `__len__()` method returns the length of the dataset, which is the number of samples in `X`.

The `__getitem__()` method takes an index idx as input and returns the idx-th sample from the dataset as a tuple containing the input and output data. The input data is accessed from self.X and the output data is accessed from `self.y`.

In [ ]:
class ColorDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

The next thing we have to do is define a PyTorch neural network model class named ColorClassifier, which inherits from nn.Module. The nn.Module is a base class in PyTorch for all neural network modules, and it provides several useful methods and attributes for constructing and manipulating neural networks.

The ColorClassifier class takes two arguments in its constructor: input_dim and num_classes. input_dim specifies the number of features in the input data, and num_classes specifies the number of output classes or labels in the classification task.

In the `__init__` method, the model architecture is defined using fully connected layers (nn.Linear()) with ReLU activation function (torch.relu()). Specifically, the model has three fully connected layers: self.fc1, self.fc2, and self.fc3, which have 64, 32, and num_classes number of neurons respectively. The `super()` function is used to call the constructor of the parent class (nn.Module) to initialize the model.

The `forward()` method specifies the forward pass computation of the neural network, which takes the input tensor x and applies the linear transformation with the weight and bias parameters learned during training. The `torch.relu()` activation function is applied after each fully connected layer, except for the output layer. Finally, the output tensor is returned.

During the training phase, given an input tensor, the `forward()` method will compute the output of the neural network, which can be compared to the target output to compute the loss and update the parameters of the model via backpropagation.

In [ ]:
class ColorClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ColorClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Now we define a function to help us read the CSV data. The function takes a single argument csv_file, which is the path to the CSV file containing the color data. The CSV file is assumed to have the following columns: "Red (8 bit)", "Green (8 bit)", "Blue (8 bit)", and "Name". The "Red (8 bit)", "Green (8 bit)", and "Blue (8 bit)" columns contain the RGB color values for each data point, and the "Name" column contains the corresponding color name or label.

The function uses the pandas library to read the CSV file into a DataFrame object df. Then, it extracts the "Red (8 bit)", "Green (8 bit)", and "Blue (8 bit)" columns from the DataFrame using df[["Red (8 bit)", "Green (8 bit)", "Blue (8 bit)"]], and converts them to a numpy array using the .values attribute. These values are then normalized by dividing by 255 to obtain values in the range [0,1]. The resulting array X contains the input data.

Similarly, the function extracts the "Name" column from the DataFrame using df["Name"], converts it to a numpy array using the `.values` attribute, and assigns it to the variable y. The resulting array y contains the corresponding output data or labels.

Finally, the function returns the input and output arrays as a tuple (X, y). This function can be used to preprocess the input and output data for a machine learning model that expects input RGB values and corresponding color names as labels.

In [ ]:
def read_color_data(csv_file):
    df = pd.read_csv(csv_file)
    X = df[["Red (8 bit)", "Green (8 bit)", "Blue (8 bit)"]].values / 255.0
    y = df["Name"].values
    return X, y

Following up, we define a function takes five arguments: model is the PyTorch neural network model to be trained, dataloader is the DataLoader object containing the training dataset, criterion is the loss function used for training the model, optimizer is the optimizer used to update the model parameters, and device is the device on which to perform the computation (e.g., "cpu" or "cuda").

The function first sets the model to "train" mode using `model.train()`. Then, it initializes a variable running_loss to zero, which will be used to accumulate the total loss over all the batches.

Next, the function iterates over the dataloader object, which returns batches of input and output data. For each batch, the inputs and labels are moved to the specified device using .to(device) method to ensure the computation is performed on the correct device. The optimizer gradients are reset to zero using optimizer.zero_grad(). The model is then applied to the input data to obtain the predicted outputs using outputs = model(inputs). The loss between the predicted outputs and the true labels is computed using the specified criterion function with loss = criterion(outputs, labels). The gradients of the loss with respect to the model parameters are computed using loss.backward(), and the optimizer is used to update the model parameters using `optimizer.step()`. The loss for the current batch is added to the running loss using running_loss += loss.item().

Finally, the function returns the average loss over all batches, which is computed by dividing the running loss by the number of batches using running_loss / len(dataloader). This value is typically used to track the training progress of the model.

In [ ]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

In this part we define a function named `test_model()` that evaluates the performance of a PyTorch neural network model using a given dataset and criterion.

The function takes four arguments: model is the PyTorch neural network model to be evaluated, dataloader is the DataLoader object containing the test dataset, criterion is the loss function used for evaluating the model, and device is the device on which to perform the computation (e.g., "cpu" or "cuda").

The function first sets the model to "eval" mode using model.eval(), which disables dropout and batch normalization layers. Then, it initializes a variable running_loss to zero, which will be used to accumulate the total loss over all the batches.

Next, the function iterates over the dataloader object, which returns batches of input and output data. For each batch, the inputs and labels are moved to the specified device using .to(device) method to ensure the computation is performed on the correct device. The model is then applied to the input data to obtain the predicted outputs using outputs = model(inputs). The loss between the predicted outputs and the true labels is computed using the specified criterion function with loss = criterion(outputs, labels). Since we are evaluating the model in this case, we don't need to compute gradients, so we set torch.no_grad() context manager to temporarily disable gradient computation, and then add the loss for the current batch to the running loss using running_loss += loss.item().

In [ ]:
def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    return running_loss / len(dataloader)

Finally here we train a PyTorch neural network to classify colors using RGB values, saves the trained model, and saves the label encoder. It reads the color data from a CSV file, encodes color names using scikit-learn's LabelEncoder, and splits the data into training and test sets. The model is defined as an instance of the `ColorClassifier` class and trained using the `train_model()` and `test_model()` functions. The trained model and label encoder are saved to files, and the function prints out messages indicating that they have been saved.

In [ ]:
X, y = read_color_data(csv_file_path)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

input_dim = X.shape[1]
num_classes = len(np.unique(y_encoded))

model = ColorClassifier(input_dim, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_dataset = ColorDataset(X_train, y_train)
test_dataset = ColorDataset(X_test, y_test)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for _ in range(num_epochs):
    train_model(model, train_dataloader, criterion, optimizer, device)
    test_model(model, test_dataloader, criterion, device)

torch.save(model.state_dict(), color_classifier_path)
with open(label_encoder_path, 'wb') as f:
    pickle.dump(label_encoder, f)

print(f"Model {color_classifier_path} was created.")
print(f"Label encoder {label_encoder_path} was created.")